In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

from collections import OrderedDict

import sys
sys.path.append("..")
import utilities, ontology

# Load brain and text data

In [2]:
act_bin = utilities.load_coordinates().astype(float)
print("Document N={}, Structure N={}".format(act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=118


In [3]:
version = 190325
dtm_bin = utilities.load_doc_term_matrix(version=version, binarize=True)
print("Document N={}, Term N={}".format(dtm_bin.shape[0], dtm_bin.shape[1]))

Document N=18155, Term N=4107


In [4]:
frameworks = ["data-driven", "rdoc", "dsm"]
list_suffixes = ["", "_opsim", "_opsim"]
circuit_suffixes = ["_lr", "", ""]
lists, circuits = {}, {}
for fw, list_suffix, circuit_suffix in zip(frameworks, list_suffixes, circuit_suffixes):
    lists[fw], circuits[fw] = utilities.load_framework(fw, suffix=list_suffix, clf=circuit_suffix)

In [5]:
scores = {fw: utilities.score_lists(lists[fw], dtm_bin, label_var="DOMAIN") for fw in frameworks}

In [6]:
words = []
for fw in frameworks:
    words += list(lists[fw]["TOKEN"])
words = sorted(list(set(words)))
structures = list(act_bin.columns)
print("Term N={}, Structure N={}".format(len(words), len(structures)))

Term N=279, Structure N=118


In [7]:
domains = {fw: list(OrderedDict.fromkeys(lists[fw]["DOMAIN"])) for fw in frameworks}

In [8]:
pmids = act_bin.index.intersection(scores["rdoc"].index).intersection(scores["dsm"].index)
len(pmids)

18155

In [9]:
for fw in frameworks:
    scores[fw] = scores[fw].loc[pmids]

In [10]:
dtm_bin = dtm_bin.loc[pmids, words]
act_bin = act_bin.loc[pmids, structures]

# Load frameworks

In [11]:
systems = {}
for fw in frameworks:
    fw_df = pd.DataFrame(0.0, index=words+structures, columns=domains[fw])
    for dom in domains[fw]:
        for word in lists[fw].loc[lists[fw]["DOMAIN"] == dom, "TOKEN"]:
            fw_df.loc[word, dom] = 1.0
        for struct in structures:
            fw_df.loc[struct, dom] = circuits[fw].loc[struct, dom]
    fw_df[fw_df > 0.0] = 1.0
    systems[fw] = fw_df

# Similarity of RDoC and data-driven systems

## Observed values

In [12]:
from scipy.spatial.distance import dice, cdist

In [13]:
def compute_sim_obs(fw):
    sims = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            sims.loc[k,r] = 1.0 - dice(systems["data-driven"][k], systems[fw][r])
    return sims

In [14]:
sims = compute_sim_obs("rdoc")
sims

,NEGATIVE_VALENCE,POSITIVE_VALENCE,AROUSAL_REGULATION,SOCIAL_PROCESSES,COGNITIVE_SYSTEMS,SENSORIMOTOR_SYSTEMS
MEMORY,0.276596,0.0449438,0.263736,0.268041,0,0
REWARD,0.375,0.56,0.207792,0.0722892,0,0
COGNITION,0.26,0.273684,0.123711,0.038835,0.117647,0.04
VISION,0.04,0,0.0206186,0.213592,0.196078,0
MANIPULATION,0,0,0,0,0.485437,0.653465
LANGUAGE,0.0792079,0,0.142857,0.153846,0,0.118812


## Null distribution

In [15]:
def compute_sim_null(fw, n_iter=10000):
    sims_null = np.empty((len(domains["data-driven"]), len(domains[fw]), n_iter))
    for n in range(n_iter):
        null = np.random.choice(words+structures, 
                                size=len(words+structures), replace=False)
        sims_null[:,:,n] = 1.0 - cdist(systems["data-driven"].loc[null].values.T, 
                                       systems[fw].values.T, metric="dice")
        if n % (float(n_iter) / 10.0) == 0:
            print("Iteration {}".format(n))
    return sims_null

In [16]:
sims_null = compute_sim_null("rdoc")

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


## False discovery rate

In [17]:
from statsmodels.stats.multitest import multipletests

In [18]:
def compute_sim_fdr(fw, sims, sims_null):
    n_iter = sims_null.shape[2]
    pvals = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for i, k in enumerate(domains["data-driven"]):
        for j, r in enumerate(domains[fw]):
            pvals.loc[k,r] = np.sum(sims_null[i,j,:] > sims.loc[k,r]) / float(n_iter)
    fdrs = multipletests(pvals.values.ravel(), method="fdr_bh")[1]
    fdrs = np.reshape(fdrs, pvals.shape)
    fdrs = pd.DataFrame(fdrs, index=domains["data-driven"], columns=domains[fw])
    return fdrs

In [19]:
fdrs = compute_sim_fdr("rdoc", sims, sims_null)
fdrs

,NEGATIVE_VALENCE,POSITIVE_VALENCE,AROUSAL_REGULATION,SOCIAL_PROCESSES,COGNITIVE_SYSTEMS,SENSORIMOTOR_SYSTEMS
MEMORY,0.00102857,0.9996,0.00225,0.00102857,0.9996,0.9996
REWARD,0,0,0.00648,0.9074,0.9996,0.9996
COGNITION,0.0056,0.00102857,0.85272,0.9996,0.9074,0.9996
VISION,0.9996,0.9996,0.9996,0.0481091,0.0975,0.9996
MANIPULATION,0.9996,0.9996,0.9996,0.9996,0,0
LANGUAGE,0.9996,0.9996,0.599657,0.54,0.9996,0.9074


In [20]:
def compute_sim_star(fw, fdrs):
    stars = pd.DataFrame("", index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            fdr = fdrs.loc[k,r]
            if fdr < 0.05:
                stars.loc[k,r] = "*"
            if fdr < 0.01:
                stars.loc[k,r] = "**"
            if fdr < 0.001:
                stars.loc[k,r] = "***"
    return stars

In [21]:
stars = compute_sim_star("rdoc", fdrs)
stars

,NEGATIVE_VALENCE,POSITIVE_VALENCE,AROUSAL_REGULATION,SOCIAL_PROCESSES,COGNITIVE_SYSTEMS,SENSORIMOTOR_SYSTEMS
MEMORY,**,,**,**,,
REWARD,***,***,**,,,
COGNITION,**,**,,,,
VISION,,,,*,,
MANIPULATION,,,,,***,***
LANGUAGE,,,,,,


## Weights for figure

In [22]:
sims[fdrs < 0.05] * 50

,NEGATIVE_VALENCE,POSITIVE_VALENCE,AROUSAL_REGULATION,SOCIAL_PROCESSES,COGNITIVE_SYSTEMS,SENSORIMOTOR_SYSTEMS
MEMORY,13.8298,NaN,13.1868,13.4021,NaN,NaN
REWARD,18.75,28,10.3896,NaN,NaN,NaN
COGNITION,13,13.6842,NaN,NaN,NaN,NaN
VISION,NaN,NaN,NaN,10.6796,NaN,NaN
MANIPULATION,NaN,NaN,NaN,NaN,24.2718,32.6733
LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN


# Similarity of DSM and data-driven systems

## Observed values

In [23]:
sims = compute_sim_obs("dsm")
sims

,DEPRESSIVE,ANXIETY,TRAUMA_STRESSOR,OBSESSIVE_COMPULSIVE,DISRUPTIVE,SUBSTANCE,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR
MEMORY,0.0714286,0.245614,0.313725,0,0,0,0,0,0.08
REWARD,0.047619,0,0.0540541,0.171429,0.368421,0.166667,0,0,0.111111
COGNITION,0.0645161,0.126984,0.0350877,0.0727273,0.0344828,0,0,0,0.0714286
VISION,0,0,0,0,0,0,0.263736,0,0
MANIPULATION,0,0,0,0,0,0.0350877,0.23913,0.196721,0.0350877
LANGUAGE,0,0,0,0.0357143,0,0,0.195652,0.0983607,0


## Null distribution

In [24]:
sims_null = compute_sim_null("dsm")

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


## False discovery rate

In [25]:
fdrs = compute_sim_fdr("dsm", sims, sims_null)
fdrs

,DEPRESSIVE,ANXIETY,TRAUMA_STRESSOR,OBSESSIVE_COMPULSIVE,DISRUPTIVE,SUBSTANCE,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR
MEMORY,0.837635,0.0036,0,0.970412,0.970412,0.970412,0.9968,0.970412,0.510508
REWARD,0.9363,0.970412,0.70335,0.03105,0,0.03105,0.985188,0.970412,0.17766
COGNITION,0.970412,0.279327,0.970412,0.563914,0.970412,0.970412,0.9968,0.970412,0.65556
VISION,0.970412,0.970412,0.970412,0.970412,0.970412,0.970412,0.0054,0.970412,0.970412
MANIPULATION,0.970412,0.970412,0.970412,0.970412,0.970412,0.970412,0.0297,0.01404,0.970412
LANGUAGE,0.970412,0.970412,0.970412,0.970412,0.970412,0.970412,0.1572,0.510508,0.970412


In [26]:
stars = compute_sim_star("dsm", fdrs)
stars

,DEPRESSIVE,ANXIETY,TRAUMA_STRESSOR,OBSESSIVE_COMPULSIVE,DISRUPTIVE,SUBSTANCE,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR
MEMORY,,**,***,,,,,,
REWARD,,,,*,***,*,,,
COGNITION,,,,,,,,,
VISION,,,,,,,**,,
MANIPULATION,,,,,,,*,*,
LANGUAGE,,,,,,,,,


## Weights for figure

In [27]:
sims[fdrs < 0.05] * 50

,DEPRESSIVE,ANXIETY,TRAUMA_STRESSOR,OBSESSIVE_COMPULSIVE,DISRUPTIVE,SUBSTANCE,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR
MEMORY,NaN,12.2807,15.6863,NaN,NaN,NaN,NaN,NaN,NaN
REWARD,NaN,NaN,NaN,8.57143,18.4211,8.33333,NaN,NaN,NaN
COGNITION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VISION,NaN,NaN,NaN,NaN,NaN,NaN,13.1868,NaN,NaN
MANIPULATION,NaN,NaN,NaN,NaN,NaN,NaN,11.9565,9.83607,NaN
LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
